# **Про использование кавычек ( ' ) и ( " )**

**В Python словарях `(dict)`:**

Можно использовать как одинарные `(')`, так и двойные `(")` кавычки для ключей и значений, всё равно:

In [3]:
person = { 'name': 'Иван',
            "age": 30,
            'city': "Москва"
         }

print(person)

{'name': 'Иван', 'age': 30, 'city': 'Москва'}


Python воспринимает `'name'` и `"name"` одинаково. Главное — быть последовательным и придерживаться читаемости.

**В JSON:**

JSON строго требует двойные кавычки `(")`, и любые одинарные кавычки вызовут ошибку:

Неправильно:

`{ 'name': 'Иван' }`

Правильно:

`{ "name": "Иван" }`

Это касается и ключей, и строковых значений. Одинарные кавычки невалидны по спецификации JSON.

**Как работать с JSON в Python:**

Python-переменные с одинарными кавычками можно сериализовать в JSON через

`json.dumps()`, и Python сам приведёт всё к нужному формату (т.е. поставит двойные кавычки):


In [1]:
import json

person = {'name': 'Иван', 'age': 30}
json_str = json.dumps(person)
print(json_str)  # {"name": "\u0418\u0432\u0430\u043d", "age": 30}

{"name": "\u0418\u0432\u0430\u043d", "age": 30}


In [2]:
json_str_2 = json.dumps(person, ensure_ascii=False)
print(json_str_2)  # {"name": "Иван", "age": 30}

{"name": "Иван", "age": 30}


# **json.dumps**

Когда происходит считывание JSON полей и запись во фрейм, то у них меняются кавычки с двойных на одинарные, поля превращаются в списки словарей. Поэтому, перед записью в таблицу Hadoop, полям необходимо вернуть правильный вид JSON.

**Поэтому:**

`df_result[col] = df_result[col].apply(lambda x: json.dumps(x, ensure_ascii=False))`

Преобразует каждый элемент в столбце `col` в строку `JSON`.

То есть превращает, например, `{'key': 'значение'}` в `'{"key": "значение"}'`.

**Зачем:**

Нужно, если планируется:

Сохранять DataFrame в `CSV`, `JSON` или базу данных, где вложенные структуры `(dict, list и т.д.)` не поддерживаются.

Передавать данные через `API` или в систему, где нужны только строки.

In [ ]:
import pandas as pd
import json

df_result = pd.DataFrame({
    'info': [{'name': 'Аня'}, {'name': 'Иван'}],
    'data': [[1, 2], [3, 4]]
})

list_undecompose_add = ['info', 'data']

for col in list_undecompose_add:
    df_result[col] = df_result[col].apply(lambda x: json.dumps(x, ensure_ascii=False))

df_result

#Теперь ячейки — это строки, а не объекты. Это полезно, например, перед экспортом в CSV или базу данных.

,info,data
0,"{""name"": ""Аня""}","[1, 2]"
1,"{""name"": ""Иван""}","[3, 4]"


In [ ]:
x = {'name': 'Аня'}
json.dumps(x, ensure_ascii=False)
# → '{"name": "Аня"}' (строка)

'{"name": "Аня"}'

In [ ]:
df = pd.DataFrame({'col': [{'a': 1}, {'b': 2}]})
df.to_csv('file.csv')

'''
Результат в файле:

col
"{'a': 1}"
"{'b': 2}"

Это невалидный JSON и потом будет трудно обратно прочитать как объект.
Строки получатся с одинарными кавычками и их нужно будет ещё "чинить".
'''

In [ ]:
df['col'] = df['col'].apply(lambda x: json.dumps(x, ensure_ascii=False))
df.to_csv('file.csv')

'''
Результат в файле:

col
{"a": 1}
{"b": 2}

Теперь это валидный JSON, легко читается обратно:

import json
json.loads('{"a": 1}')  # → {'a': 1}
'''

# **Пример ошибок**

In [4]:
import json

bad_json = "{'name': 'Иван'}"  # ← одинарные кавычки!

data = json.loads(bad_json)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [6]:
good_json = '{"name": "Иван"}'
data = json.loads(good_json)
data

{'name': 'Иван'}